# 🚀 Complete LangGraph Tutorial: From Basics to Advanced Applications

## 📖 Comprehensive Guide for Building Stateful AI Agents

Welcome to the most comprehensive LangGraph tutorial! This notebook will take you from complete beginner to building sophisticated AI agents step by step.

### 🎯 What You'll Learn

By the end of this tutorial, you will:
- ✅ Understand LangGraph's core concepts (Nodes, Edges, State)
- ✅ Build your first simple chatbot
- ✅ Add memory and persistence to conversations
- ✅ Implement tool calling and external integrations
- ✅ Create human-in-the-loop workflows
- ✅ Build multi-agent systems
- ✅ Handle complex state management
- ✅ Deploy production-ready applications

### 📚 Based on Official LangGraph Documentation

This tutorial follows the [official LangGraph documentation](https://langchain-ai.github.io/langgraph/) and incorporates best practices from the LangGraph team.

---

## 📋 Table of Contents

1. **[Setup & Installation](#setup)**
2. **[Part 1: Understanding LangGraph Fundamentals](#part1)**
3. **[Part 2: Building Your First Simple Agent](#part2)**
4. **[Part 3: Adding Memory with Checkpointing](#part3)**
5. **[Part 4: Tool Integration & External APIs](#part4)**
6. **[Part 5: Human-in-the-Loop Workflows](#part5)**
7. **[Part 6: Advanced State Management](#part6)**
8. **[Part 7: Multi-Agent Systems](#part7)**
9. **[Part 8: Real-World Use Cases](#part8)**
10. **[Part 9: Production Deployment](#part9)**

Let's begin this exciting journey! 🌟


## 🔧 Setup & Installation {#setup}

Before we dive into LangGraph, let's set up our environment properly.

### Prerequisites

1. **Python 3.8+** installed on your system
2. **API Keys** for LLM providers (we'll use OpenAI in this tutorial)
3. **Basic Python knowledge** - understanding of functions, classes, and dictionaries

### What is LangGraph?

**LangGraph** is a library for building **stateful, multi-actor applications** with LLMs. It extends LangChain with the ability to coordinate multiple chains (or actors) across multiple steps of computation in a **cyclic** manner.

Key features:
- 🔄 **Cyclic workflows** (not just linear chains)
- 💾 **Persistent state** across interactions
- 🎯 **Conditional routing** between different paths
- 🔧 **Human-in-the-loop** capabilities
- 📊 **Built-in observability** with LangSmith

Think of it as a way to build AI agents that can:
- Remember previous conversations
- Make decisions about what to do next
- Use tools and external APIs
- Involve humans when needed
- Handle complex, multi-step workflows


In [ ]:
# Install required packages
%pip install -q langgraph langsmith langchain-openai python-dotenv

# Optional: for visualization
%pip install -q matplotlib graphviz


In [2]:
import os
import getpass
from typing import Annotated, Dict, List, Any
from typing_extensions import TypedDict

# Set up environment variables
def setup_environment():
    """Setup API keys for the tutorial"""
    
    # OpenAI API Key
    if not os.environ.get("OPENAI_API_KEY"):
        openai_key = getpass.getpass("Enter your OpenAI API Key: ")
        os.environ["OPENAI_API_KEY"] = openai_key
    
    # Optional: LangSmith for observability (highly recommended)
    if not os.environ.get("LANGSMITH_API_KEY"):
        print("LangSmith setup (optional but recommended for debugging):")
        langsmith_key = getpass.getpass("Enter your LangSmith API Key (or press Enter to skip): ")
        if langsmith_key:
            os.environ["LANGSMITH_API_KEY"] = langsmith_key
            os.environ["LANGCHAIN_TRACING_V2"] = "true"
            os.environ["LANGCHAIN_PROJECT"] = "LangGraph-Tutorial"
            print("✅ LangSmith tracing enabled!")
        else:
            print("⚠️ Skipping LangSmith setup")
    
    print("🚀 Environment setup complete!")

# Run setup
setup_environment()


LangSmith setup (optional but recommended for debugging):
✅ LangSmith tracing enabled!
🚀 Environment setup complete!


## 🧠 Part 1: Understanding LangGraph Fundamentals {#part1}

Before we build anything, let's understand the core concepts that make LangGraph powerful.

### 🔑 Core Concepts

#### 1. **State** 
- The "memory" of your application
- Shared data structure that persists across all steps
- Can contain messages, variables, flags, or any data you need

#### 2. **Nodes**
- Individual functions or operations in your workflow
- Each node receives the current state and returns updates
- Think of them as "workers" that do specific tasks

#### 3. **Edges** 
- Connections between nodes that define the flow
- Can be simple (A → B) or conditional (A → B or C based on logic)

#### 4. **Graph**
- The complete workflow combining nodes and edges
- Defines how your AI agent behaves and makes decisions

### 🎯 Simple Mental Model

Think of LangGraph like a **flowchart for AI agents**:

```
[User Input] → [AI Thinks] → [Uses Tool?] 
                    ↓              ↓
               [Respond]      [Call Tool] → [AI Thinks] → [Respond]
```

But unlike a simple flowchart, LangGraph can:
- Remember everything that happened before
- Loop back to previous steps
- Involve humans in the decision-making
- Handle complex, branching logic

Let's see this in action! 👇


## 🤖 Part 2: Building Your First Simple Agent {#part2}

Let's start with the simplest possible LangGraph application - a basic chatbot that can have a conversation.

### Step 1: Define the State

The state is like the "memory" of our agent. For a chatbot, we need to remember the conversation history.


In [3]:
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage

# Define our State - this is the "memory" of our agent
class State(TypedDict):
    # messages will store our conversation history
    # add_messages is a special function that appends new messages instead of replacing them
    messages: Annotated[list, add_messages]

print("✅ State defined!")
print("Our state has one field: 'messages' that will store the conversation history")
print("The add_messages function ensures new messages are appended, not replaced")


✅ State defined!
Our state has one field: 'messages' that will store the conversation history
The add_messages function ensures new messages are appended, not replaced


In [4]:
from langchain_openai import ChatOpenAI

# Initialize the language model
llm = ChatOpenAI(
    model="gpt-3.5-turbo",  # You can also use "gpt-4" if you have access
    temperature=0.7,        # Controls creativity (0 = deterministic, 1 = very creative)
)

print("✅ Language model initialized!")
print(f"Model: {llm.model_name}")

# Let's test it quickly
test_response = llm.invoke("Say hello in a friendly way!")
print(f"Test response: {test_response.content}")


✅ Language model initialized!
Model: gpt-3.5-turbo


Failed to multipart ingest runs: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


Test response: Hello there! How are you doing today?


Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


In [ ]:
def chatbot_node(state: State) -> Dict[str, Any]:
    """
    The main chatbot node that processes messages and generates responses.
    
    Args:
        state: Current state containing conversation history
        
    Returns:
        Dict with new messages to add to state
    """
    # Get the conversation history from state
    messages = state["messages"]
    
    # Call the LLM with the conversation history
    response = llm.invoke(messages)
    
    # Return the new message to be added to state
    return {"messages": [response]}

print("✅ Chatbot node created!")
print("This node will:")
print("1. Take the conversation history from state")
print("2. Send it to the LLM")
print("3. Return the LLM's response to be added to state")


In [ ]:
# Step 1: Create the graph builder
graph_builder = StateGraph(State)

# Step 2: Add our chatbot node
graph_builder.add_node("chatbot", chatbot_node)

# Step 3: Define the flow
# START -> chatbot -> END
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)

# Step 4: Compile the graph
simple_chatbot = graph_builder.compile()

print("✅ Simple chatbot graph created!")
print("Flow: START → chatbot → END")
print("Ready to chat! 🎉")


In [ ]:
# Test our simple chatbot
def test_simple_chatbot():
    print("🤖 Testing Simple Chatbot")
    print("=" * 40)
    
    # Create initial state with a user message
    initial_state = {
        "messages": [HumanMessage(content="Hello! My name is Alex. What's your name?")]
    }
    
    # Run the chatbot
    result = simple_chatbot.invoke(initial_state)
    
    # Print the conversation
    for i, message in enumerate(result["messages"]):
        if isinstance(message, HumanMessage):
            print(f"👤 Human: {message.content}")
        elif isinstance(message, AIMessage):
            print(f"🤖 AI: {message.content}")
    
    print("\n" + "=" * 40)
    return result

# Run the test
first_result = test_simple_chatbot()


In [ ]:
# Test memory issue - the chatbot won't remember the previous conversation
print("🧪 Testing Memory Issue")
print("=" * 40)

# Second message - asking about the name mentioned earlier
second_state = {
    "messages": [HumanMessage(content="What was my name again?")]
}

result2 = simple_chatbot.invoke(second_state)

for message in result2["messages"]:
    if isinstance(message, HumanMessage):
        print(f"👤 Human: {message.content}")
    elif isinstance(message, AIMessage):
        print(f"🤖 AI: {message.content}")

print("\n❌ As you can see, the chatbot doesn't remember your name!")
print("This is because each call starts with a fresh state.")
print("In the next section, we'll fix this with **memory and checkpointing**! 🧠")


In [ ]:
from langgraph.checkpoint.memory import MemorySaver
memory = MemorySaver()


print("✅ Checkpointer created!")
print("This will automatically save and restore conversation state")
print("In production, you'd use a real database file instead of ':memory:'")


In [ ]:
graph_builder_with_memory = StateGraph(State)

graph_builder_with_memory.add_node("chatbot", chatbot_node)
graph_builder_with_memory.add_edge(START, "chatbot")
graph_builder_with_memory.add_edge("chatbot", END)

chatbot_with_memory = graph_builder_with_memory.compile(checkpointer=memory)
unique_id = uuid.uuid4()
print(f"unique ID:- {unique_id}")


unique_id = "831173cc-dd33-4542-925d-7958802a975f"
def test_simple(user_input):
    
    config = {"configurable": {"thread_id":str(unique_id)}}
    result = chatbot_with_memory.invoke(
        {"messages": [HumanMessage(content=user_input)]},
        config
    )
    return result
    


In [ ]:
def test_memory_chatbot():
    print("🧠 Testing Chatbot with Memory")
    print("=" * 50)
    
    # Configuration with thread_id - this is crucial for memory!
    config = {"configurable": {"thread_id": "conversation_1"}}
    
    # First message
    print("📝 First interaction:")
    result1 = chatbot_with_memory.invoke(
        {"messages": [HumanMessage(content="Hi! My name is Alex and I love programming.")]},
        config  # Pass config as second parameter!
    )
    
    for message in result1["messages"]:
        if isinstance(message, HumanMessage):
            print(f"👤 Human: {message.content}")
        elif isinstance(message, AIMessage):
            print(f"🤖 AI: {message.content}")
    
    print("\n📝 Second interaction (same thread_id):")
    result2 = chatbot_with_memory.invoke(
        {"messages": [HumanMessage(content="What's my name and what do I love?")]},
        config  # Same config = same memory!
    )
    
    # Only show the new messages
    new_messages = result2["messages"][len(result1["messages"]):]
    for message in new_messages:
        if isinstance(message, HumanMessage):
            print(f"👤 Human: {message.content}")
        elif isinstance(message, AIMessage):
            print(f"🤖 AI: {message.content}")
    
    print("\n✅ Success! The chatbot remembered both your name and interest!")
    return result2

# Test the memory
memory_result = test_memory_chatbot()


In [ ]:
# Test with a different thread_id
print("🔄 Testing Different Thread ID")
print("=" * 40)

# Different thread_id = fresh conversation
different_config = {"configurable": {"thread_id": "conversation_2"}}

result_different = chatbot_with_memory.invoke(
    {"messages": [HumanMessage(content="What's my name?")]},
    different_config  # Different thread_id!
)

for message in result_different["messages"]:
    if isinstance(message, HumanMessage):
        print(f"👤 Human: {message.content}")
    elif isinstance(message, AIMessage):
        print(f"🤖 AI: {message.content}")

print("\n🎯 Key Insight:")
print("- Same thread_id = shared memory")
print("- Different thread_id = separate conversations")
print("- This allows multiple users or conversation contexts!")


In [ ]:
# Inspect the state of our first conversation
config1 = {"configurable": {"thread_id": "conversation_1"}}
state_snapshot = chatbot_with_memory.get_state(config1)

print("📊 State Inspection")
print("=" * 30)
print(f"Number of messages: {len(state_snapshot.values['messages'])}")
print(f"Next node to execute: {state_snapshot.next}")
print(f"Thread ID: {state_snapshot.config['configurable']['thread_id']}")

print("\n💬 Full conversation history:")
for i, message in enumerate(state_snapshot.values["messages"], 1):
    if isinstance(message, HumanMessage):
        print(f"{i}. 👤 Human: {message.content}")
    elif isinstance(message, AIMessage):
        print(f"{i}. 🤖 AI: {message.content[:100]}...")  # Truncate for readability


In [ ]:
from langchain_core.tools import tool
import math

@tool
def calculator(expression: str) -> str:
    """
    Evaluate a mathematical expression safely.
    
    Args:
        expression: A mathematical expression to evaluate (e.g., "2 + 3 * 4")
        
    Returns:
        The result of the calculation
    """
    try:
        # Safe evaluation of mathematical expressions
        # Only allow basic math operations
        allowed_names = {
            k: v for k, v in math.__dict__.items() if not k.startswith("__")
        }
        allowed_names.update({"abs": abs, "round": round})
        
        result = eval(expression, {"__builtins__": {}}, allowed_names)
        return f"The result of {expression} is {result}"
    except Exception as e:
        return f"Error calculating {expression}: {str(e)}"

@tool  
def get_current_time() -> str:
    """Get the current time."""
    from datetime import datetime
    return f"Current time is: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"

# Test our tools
print("🧮 Testing Calculator Tool:")
print(calculator.invoke({"expression": "2 + 3 * 4"}))
print(calculator.invoke({"expression": "sqrt(16) + 5"}))

print("\n⏰ Testing Time Tool:")
print(get_current_time.invoke({}))


In [ ]:
from langgraph.prebuilt import ToolNode
from langchain_core.messages import ToolMessage

# Step 1: Create LLM with tools
tools = [calculator, get_current_time]
llm_with_tools = llm.bind_tools(tools)

# Step 2: Create nodes
def chatbot_with_tools(state: State) -> Dict[str, Any]:
    """Chatbot that can use tools"""
    messages = state["messages"]
    response = llm_with_tools.invoke(messages)
    return {"messages": [response]}

# Step 3: Create tool node using LangGraph's prebuilt ToolNode
tool_node = ToolNode(tools)

print("✅ Tool-enabled chatbot components created!")
print("- LLM knows about our tools")
print("- Chatbot node can generate tool calls") 
print("- Tool node can execute the tools")


In [ ]:
from typing import Literal

def should_continue(state: State) -> Literal["tools", "__end__"]:
    """
    Determine if we should use tools or end the conversation.
    
    Returns:
        "tools" if the last message has tool calls
        "__end__" if we should end
    """
    messages = state["messages"]
    last_message = messages[-1]
    
    # If the last message has tool calls, we should use tools
    if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
        return "tools"
    else:
        return "__end__"

print("✅ Conditional logic created!")
print("This function decides whether to:")
print("- Use tools (if LLM made tool calls)")
print("- End conversation (if LLM gave final response)")


In [ ]:
# Create the graph
tool_graph_builder = StateGraph(State)

# Add nodes
tool_graph_builder.add_node("chatbot", chatbot_with_tools)
tool_graph_builder.add_node("tools", tool_node)

# Add edges
tool_graph_builder.add_edge(START, "chatbot")

# Add conditional edges
tool_graph_builder.add_conditional_edges(
    "chatbot",  # From chatbot node
    should_continue,  # Use this function to decide
    {
        "tools": "tools",    # If should_continue returns "tools", go to tools node
        "__end__": END       # If should_continue returns "__end__", end the graph
    }
)

# After tools, always go back to chatbot
tool_graph_builder.add_edge("tools", "chatbot")

# Compile with memory
tool_chatbot = tool_graph_builder.compile(checkpointer=checkpointer)

print("✅ Tool-enabled chatbot with memory created!")
print("Flow: START → chatbot → [tools OR end] → (if tools) → chatbot → ...")


In [ ]:
def test_tool_chatbot():
    print("🛠️ Testing Tool-Enabled Chatbot")
    print("=" * 50)
    
    config = {"configurable": {"thread_id": "tool_test_1"}}
    
    # Test 1: Math calculation
    print("📝 Test 1: Math Calculation")
    result1 = tool_chatbot.invoke(
        {"messages": [HumanMessage(content="What's 15 * 7 + sqrt(144)?")]},
        config
    )
    
    for message in result1["messages"]:
        if isinstance(message, HumanMessage):
            print(f"👤 Human: {message.content}")
        elif isinstance(message, AIMessage):
            if hasattr(message, 'tool_calls') and message.tool_calls:
                print(f"🤖 AI: [Calling tool: {message.tool_calls[0]['name']}]")
            else:
                print(f"🤖 AI: {message.content}")
        elif isinstance(message, ToolMessage):
            print(f"🔧 Tool: {message.content}")
    
    print("\n📝 Test 2: Current Time")
    result2 = tool_chatbot.invoke(
        {"messages": [HumanMessage(content="What time is it right now?")]},
        config
    )
    
    # Show only new messages
    new_messages = result2["messages"][len(result1["messages"]):]
    for message in new_messages:
        if isinstance(message, HumanMessage):
            print(f"👤 Human: {message.content}")
        elif isinstance(message, AIMessage):
            if hasattr(message, 'tool_calls') and message.tool_calls:
                print(f"🤖 AI: [Calling tool: {message.tool_calls[0]['name']}]")
            else:
                print(f"🤖 AI: {message.content}")
        elif isinstance(message, ToolMessage):
            print(f"🔧 Tool: {message.content}")
    
    print("\n✅ Tool integration working perfectly!")
    return result2

# Test the tool-enabled chatbot
tool_result = test_tool_chatbot()


In [ ]:
# Enhanced State with human assistance flag
class HumanLoopState(TypedDict):
    messages: Annotated[list, add_messages]
    ask_human: bool  # Flag to request human help

@tool
def request_human_help(question: str) -> str:
    """
    Request help from a human supervisor.
    
    Args:
        question: The question or situation where human help is needed
        
    Returns:
        Confirmation that help has been requested
    """
    return f"Human help requested for: {question}"

# Add the new tool to our toolkit
human_tools = [calculator, get_current_time, request_human_help]
llm_with_human_tools = llm.bind_tools(human_tools)

print("✅ Human-in-the-loop tools created!")
print("The agent can now request human assistance when needed")


In [ ]:
def chatbot_with_human_help(state: HumanLoopState) -> Dict[str, Any]:
    """Chatbot that can request human help"""
    messages = state["messages"]
    response = llm_with_human_tools.invoke(messages)
    
    # Check if the agent requested human help
    ask_human = False
    if hasattr(response, 'tool_calls') and response.tool_calls:
        for tool_call in response.tool_calls:
            if tool_call['name'] == 'request_human_help':
                ask_human = True
                break
    
    return {"messages": [response], "ask_human": ask_human}

def human_node(state: HumanLoopState) -> Dict[str, Any]:
    """
    Human intervention node - this is where humans can provide input
    """
    # In a real application, this would wait for human input
    # For this demo, we'll simulate human response
    last_message = state["messages"][-1]
    
    if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
        tool_call = last_message.tool_calls[0]
        if tool_call['name'] == 'request_human_help':
            # Simulate human response
            human_response = "I've reviewed your question. Please proceed with the calculation and provide a detailed explanation."
            
            # Create a tool message with the human's response
            tool_message = ToolMessage(
                content=human_response,
                tool_call_id=tool_call['id']
            )
            return {"messages": [tool_message], "ask_human": False}
    
    return {"ask_human": False}

print("✅ Human-in-the-loop nodes created!")
print("- chatbot_with_human_help: Can request human assistance")
print("- human_node: Handles human intervention")


In [ ]:
def route_human_loop(state: HumanLoopState) -> Literal["human", "tools", "__end__"]:
    """Route based on whether human help is needed or tools should be called"""
    
    if state.get("ask_human", False):
        return "human"
    
    messages = state["messages"]
    last_message = messages[-1]
    
    # Check for tool calls (excluding request_human_help)
    if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
        for tool_call in last_message.tool_calls:
            if tool_call['name'] != 'request_human_help':
                return "tools"
    
    return "__end__"

# Build the graph
human_loop_builder = StateGraph(HumanLoopState)

# Add nodes
human_loop_builder.add_node("chatbot", chatbot_with_human_help)
human_loop_builder.add_node("tools", ToolNode(human_tools))
human_loop_builder.add_node("human", human_node)

# Add edges
human_loop_builder.add_edge(START, "chatbot")

# Add conditional routing
human_loop_builder.add_conditional_edges(
    "chatbot",
    route_human_loop,
    {
        "human": "human",
        "tools": "tools", 
        "__end__": END
    }
)

# After human or tools, go back to chatbot
human_loop_builder.add_edge("human", "chatbot")
human_loop_builder.add_edge("tools", "chatbot")

# Compile with interrupt before human node
human_loop_chatbot = human_loop_builder.compile(
    checkpointer=checkpointer,
    interrupt_before=["human"]  # This pauses execution before human node
)

print("✅ Human-in-the-loop chatbot created!")
print("Key feature: interrupt_before=['human'] pauses for human input")


In [ ]:
def test_human_loop():
    print("🤝 Testing Human-in-the-Loop")
    print("=" * 40)
    
    config = {"configurable": {"thread_id": "human_loop_test"}}
    
    # Request that requires human help
    print("📝 Asking agent to request human help:")
    
    initial_result = human_loop_chatbot.invoke(
        {"messages": [HumanMessage(content="I need help with a complex decision. Can you request human assistance?")]},
        config
    )
    
    for message in initial_result["messages"]:
        if isinstance(message, HumanMessage):
            print(f"👤 Human: {message.content}")
        elif isinstance(message, AIMessage):
            if hasattr(message, 'tool_calls') and message.tool_calls:
                print(f"🤖 AI: [Requesting human help]")
            else:
                print(f"🤖 AI: {message.content}")
    
    # Check if execution was interrupted
    state = human_loop_chatbot.get_state(config)
    print(f"\n🔍 Current state:")
    print(f"Next node to execute: {state.next}")
    print(f"Ask human flag: {state.values.get('ask_human', False)}")
    
    if state.next == ("human",):
        print("\n✅ Execution interrupted! Human intervention required.")
        print("In a real app, a human would now review and provide input.")
        
        # Continue execution (simulating human approval)
        print("\n▶️ Continuing execution (simulating human input)...")
        final_result = human_loop_chatbot.invoke(None, config)
        
        # Show the final messages
        new_messages = final_result["messages"][len(initial_result["messages"]):]
        for message in new_messages:
            if isinstance(message, ToolMessage):
                print(f"👨‍💼 Human: {message.content}")
            elif isinstance(message, AIMessage):
                print(f"🤖 AI: {message.content}")
    
    return final_result

# Test the human-in-the-loop functionality
human_loop_result = test_human_loop()


In [ ]:
# Advanced State for Research Assistant
class ResearchState(TypedDict):
    messages: Annotated[list, add_messages]
    research_topic: str
    findings: List[str]
    search_count: int
    confidence_score: float
    max_searches: int

# Research tools
@tool
def web_search(query: str) -> str:
    """
    Simulate a web search (in real app, use actual search API).
    
    Args:
        query: Search query
        
    Returns:
        Search results
    """
    # Simulate search results
    results = {
        "python": "Python is a high-level programming language known for its simplicity and versatility.",
        "machine learning": "Machine learning is a subset of AI that enables computers to learn from data.",
        "langgraph": "LangGraph is a library for building stateful, multi-actor applications with LLMs.",
        "default": f"Search results for '{query}': This is simulated search data with relevant information."
    }
    
    return results.get(query.lower(), results["default"])

@tool
def analyze_findings(findings_list: str) -> str:
    """
    Analyze research findings and provide insights.
    
    Args:
        findings_list: Comma-separated list of findings
        
    Returns:
        Analysis of the findings
    """
    findings = findings_list.split(',')
    analysis = f"Analysis of {len(findings)} findings: "
    
    if len(findings) >= 3:
        analysis += "Comprehensive research completed. Strong evidence base."
    elif len(findings) >= 2:
        analysis += "Good research foundation. Consider additional sources."
    else:
        analysis += "Limited research. More investigation needed."
    
    return analysis

# Research tools list
research_tools = [web_search, analyze_findings, calculator, get_current_time]
research_llm = llm.bind_tools(research_tools)

print("✅ Advanced research assistant tools created!")
print("Tools available: web_search, analyze_findings, calculator, get_current_time")


In [ ]:
def research_chatbot(state: ResearchState) -> Dict[str, Any]:
    """Research chatbot with advanced state management"""
    messages = state["messages"]
    
    # Add context about current research state
    context_message = f"""
Current research status:
- Topic: {state.get('research_topic', 'Not set')}
- Findings so far: {len(state.get('findings', []))}
- Searches performed: {state.get('search_count', 0)}/{state.get('max_searches', 5)}
- Confidence: {state.get('confidence_score', 0.0):.1f}/10.0
"""
    
    # Add context to messages for the LLM
    full_messages = messages + [HumanMessage(content=context_message)]
    response = research_llm.invoke(full_messages)
    
    return {"messages": [response]}

def research_tool_node(state: ResearchState) -> Dict[str, Any]:
    """Enhanced tool node that updates research state"""
    # Execute tools
    tool_node_result = ToolNode(research_tools)(state)
    
    # Update research-specific state
    updates = {}
    last_message = state["messages"][-1]
    
    if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
        for tool_call in last_message.tool_calls:
            if tool_call['name'] == 'web_search':
                # Increment search count
                updates["search_count"] = state.get("search_count", 0) + 1
                
                # Add to findings
                current_findings = state.get("findings", [])
                query = tool_call['args']['query']
                current_findings.append(f"Search: {query}")
                updates["findings"] = current_findings
                
                # Update confidence based on number of searches
                search_count = updates["search_count"]
                updates["confidence_score"] = min(search_count * 2.0, 10.0)
    
    # Combine tool results with state updates
    result = {"messages": tool_node_result["messages"]}
    result.update(updates)
    
    return result

def initialize_research(state: ResearchState) -> Dict[str, Any]:
    """Initialize research session if needed"""
    updates = {}
    
    if not state.get("research_topic"):
        # Extract topic from the first message
        if state["messages"]:
            first_msg = state["messages"][0].content
            updates["research_topic"] = first_msg[:100]  # First 100 chars as topic
    
    if "search_count" not in state:
        updates["search_count"] = 0
    
    if "findings" not in state:
        updates["findings"] = []
        
    if "confidence_score" not in state:
        updates["confidence_score"] = 0.0
        
    if "max_searches" not in state:
        updates["max_searches"] = 5
    
    return updates

print("✅ Research assistant nodes created!")
print("- research_chatbot: Handles conversations with research context")
print("- research_tool_node: Executes tools and updates research state") 
print("- initialize_research: Sets up initial research parameters")


In [ ]:
def research_router(state: ResearchState) -> Literal["tools", "complete", "__end__"]:
    """Advanced routing logic for research assistant"""
    
    # Check if we've reached max searches
    if state.get("search_count", 0) >= state.get("max_searches", 5):
        return "complete"
    
    # Check for tool calls
    messages = state["messages"]
    if messages:
        last_message = messages[-1]
        if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
            return "tools"
    
    return "__end__"

def complete_research(state: ResearchState) -> Dict[str, Any]:
    """Finalize research and provide summary"""
    findings = state.get("findings", [])
    confidence = state.get("confidence_score", 0.0)
    topic = state.get("research_topic", "Unknown")
    
    summary = f"""
Research Complete!

Topic: {topic}
Total findings: {len(findings)}
Confidence score: {confidence:.1f}/10.0

Summary of findings:
{chr(10).join(f"- {finding}" for finding in findings)}

Research session concluded.
"""
    
    return {"messages": [AIMessage(content=summary)]}

# Build the research assistant graph
research_builder = StateGraph(ResearchState)

# Add nodes
research_builder.add_node("initialize", initialize_research)
research_builder.add_node("chatbot", research_chatbot)
research_builder.add_node("tools", research_tool_node)
research_builder.add_node("complete", complete_research)

# Add edges
research_builder.add_edge(START, "initialize")
research_builder.add_edge("initialize", "chatbot")

# Add conditional routing
research_builder.add_conditional_edges(
    "chatbot",
    research_router,
    {
        "tools": "tools",
        "complete": "complete",
        "__end__": END
    }
)

research_builder.add_edge("tools", "chatbot")
research_builder.add_edge("complete", END)

# Compile
research_assistant = research_builder.compile(checkpointer=checkpointer)

print("✅ Advanced research assistant created!")
print("Flow: START → initialize → chatbot → [tools/complete/end] → ...")


In [ ]:
def test_research_assistant():
    print("🔬 Testing Advanced Research Assistant")
    print("=" * 50)
    
    config = {"configurable": {"thread_id": "research_session_1"}}
    
    # Start research
    result = research_assistant.invoke(
        {"messages": [HumanMessage(content="I want to research machine learning applications")]},
        config
    )
    
    print("📊 Research Session Started")
    print("-" * 30)
    
    # Show state after initialization
    state = research_assistant.get_state(config)
    print(f"Topic: {state.values.get('research_topic', 'Not set')}")
    print(f"Search count: {state.values.get('search_count', 0)}")
    print(f"Confidence: {state.values.get('confidence_score', 0.0):.1f}")
    print(f"Findings: {len(state.values.get('findings', []))}")
    
    # Continue with searches
    print("\n📝 Requesting searches...")
    result = research_assistant.invoke(
        {"messages": [HumanMessage(content="Please search for information about machine learning")]},
        config
    )
    
    # Show updated state
    state = research_assistant.get_state(config)
    print(f"\nAfter search:")
    print(f"Search count: {state.values.get('search_count', 0)}")
    print(f"Confidence: {state.values.get('confidence_score', 0.0):.1f}")
    print(f"Findings: {state.values.get('findings', [])}")
    
    # Show final messages
    print("\n💬 Final Response:")
    for message in result["messages"][-2:]:  # Show last 2 messages
        if isinstance(message, HumanMessage):
            print(f"👤 Human: {message.content}")
        elif isinstance(message, AIMessage):
            print(f"🤖 AI: {message.content[:200]}...")  # Truncate for readability
        elif isinstance(message, ToolMessage):
            print(f"🔧 Tool: {message.content[:100]}...")
    
    return result

# Test the research assistant
research_result = test_research_assistant()


In [ ]:
class MultiAgentState(TypedDict):
    messages: Annotated[list, add_messages]
    task: str
    research_data: List[str]
    analysis_results: List[str]
    final_report: str
    current_agent: str
    task_complete: bool

# Specialized agents
def researcher_agent(state: MultiAgentState) -> Dict[str, Any]:
    """Specialized research agent"""
    
    # Create research-focused prompt
    research_prompt = f"""
You are a Research Agent. Your job is to gather information about: {state.get('task', 'the given topic')}.

Current research data: {state.get('research_data', [])}

Please search for relevant information and add your findings to the research data.
Focus on factual information and reliable sources.
"""
    
    messages = state["messages"] + [HumanMessage(content=research_prompt)]
    response = research_llm.invoke(messages)
    
    return {
        "messages": [response],
        "current_agent": "researcher"
    }

def analyst_agent(state: MultiAgentState) -> Dict[str, Any]:
    """Specialized analysis agent"""
    
    research_data = state.get('research_data', [])
    analysis_prompt = f"""
You are an Analysis Agent. Your job is to analyze the research data and provide insights.

Research data to analyze:
{chr(10).join(f"- {data}" for data in research_data)}

Please provide analysis, identify patterns, and draw conclusions.
Focus on insights and implications.
"""
    
    messages = state["messages"] + [HumanMessage(content=analysis_prompt)]
    response = llm.invoke(messages)
    
    # Extract analysis from response (simplified)
    analysis_results = state.get('analysis_results', [])
    analysis_results.append(f"Analysis: {response.content[:200]}...")
    
    return {
        "messages": [response],
        "analysis_results": analysis_results,
        "current_agent": "analyst"
    }

def writer_agent(state: MultiAgentState) -> Dict[str, Any]:
    """Specialized writing agent"""
    
    research_data = state.get('research_data', [])
    analysis_results = state.get('analysis_results', [])
    
    writing_prompt = f"""
You are a Writing Agent. Create a comprehensive final report.

Research Data:
{chr(10).join(f"- {data}" for data in research_data)}

Analysis Results:
{chr(10).join(f"- {result}" for result in analysis_results)}

Please create a well-structured final report that combines the research and analysis.
"""
    
    messages = state["messages"] + [HumanMessage(content=writing_prompt)]
    response = llm.invoke(messages)
    
    return {
        "messages": [response],
        "final_report": response.content,
        "current_agent": "writer",
        "task_complete": True
    }

print("✅ Multi-agent system agents created!")
print("- Researcher Agent: Gathers information")
print("- Analyst Agent: Analyzes findings") 
print("- Writer Agent: Creates final reports")


In [ ]:
def multi_agent_router(state: MultiAgentState) -> Literal["researcher", "analyst", "writer", "__end__"]:
    """Route between different agents based on current state"""
    
    if state.get("task_complete", False):
        return "__end__"
    
    current_agent = state.get("current_agent", "")
    research_data = state.get("research_data", [])
    analysis_results = state.get("analysis_results", [])
    
    # Start with researcher
    if not current_agent:
        return "researcher"
    
    # After researcher, go to analyst if we have research data
    if current_agent == "researcher" and len(research_data) > 0:
        return "analyst"
    
    # After analyst, go to writer if we have analysis
    if current_agent == "analyst" and len(analysis_results) > 0:
        return "writer"
    
    # Continue with current agent if more work needed
    if current_agent == "researcher" and len(research_data) < 2:
        return "researcher"
    
    return "__end__"

def initialize_multi_agent(state: MultiAgentState) -> Dict[str, Any]:
    """Initialize multi-agent state"""
    updates = {}
    
    if not state.get("task") and state.get("messages"):
        # Extract task from first message
        first_msg = state["messages"][0].content
        updates["task"] = first_msg
    
    if "research_data" not in state:
        updates["research_data"] = []
    
    if "analysis_results" not in state:
        updates["analysis_results"] = []
    
    if "current_agent" not in state:
        updates["current_agent"] = ""
    
    if "task_complete" not in state:
        updates["task_complete"] = False
    
    return updates

# Enhanced tool node for multi-agent
def multi_agent_tool_node(state: MultiAgentState) -> Dict[str, Any]:
    """Tool node that updates research data"""
    tool_result = ToolNode(research_tools)(state)
    
    # Update research data if web search was performed
    updates = {}
    last_message = state["messages"][-1]
    
    if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
        for tool_call in last_message.tool_calls:
            if tool_call['name'] == 'web_search':
                research_data = state.get("research_data", [])
                query = tool_call['args']['query']
                # Add the search result to research data
                research_data.append(f"Search result for '{query}': {tool_result['messages'][-1].content}")
                updates["research_data"] = research_data
    
    result = {"messages": tool_result["messages"]}
    result.update(updates)
    return result

# Build multi-agent graph
multi_agent_builder = StateGraph(MultiAgentState)

# Add nodes
multi_agent_builder.add_node("initialize", initialize_multi_agent)
multi_agent_builder.add_node("researcher", researcher_agent)
multi_agent_builder.add_node("tools", multi_agent_tool_node)
multi_agent_builder.add_node("analyst", analyst_agent)
multi_agent_builder.add_node("writer", writer_agent)

# Add edges
multi_agent_builder.add_edge(START, "initialize")

# From initialize, route to appropriate agent
multi_agent_builder.add_conditional_edges(
    "initialize",
    multi_agent_router,
    {
        "researcher": "researcher",
        "analyst": "analyst",
        "writer": "writer",
        "__end__": END
    }
)

# Researcher can use tools or continue to next agent
multi_agent_builder.add_conditional_edges(
    "researcher",
    lambda state: "tools" if (
        state["messages"] and 
        hasattr(state["messages"][-1], 'tool_calls') and 
        state["messages"][-1].tool_calls
    ) else multi_agent_router(state),
    {
        "tools": "tools",
        "researcher": "researcher",
        "analyst": "analyst",
        "writer": "writer",
        "__end__": END
    }
)

# After tools, route to next agent
multi_agent_builder.add_conditional_edges(
    "tools",
    multi_agent_router,
    {
        "researcher": "researcher",
        "analyst": "analyst",
        "writer": "writer",
        "__end__": END
    }
)

# Analyst routes to next agent
multi_agent_builder.add_conditional_edges(
    "analyst",
    multi_agent_router,
    {
        "researcher": "researcher",
        "analyst": "analyst",
        "writer": "writer",
        "__end__": END
    }
)

# Writer completes the task
multi_agent_builder.add_edge("writer", END)

# Compile
multi_agent_system = multi_agent_builder.compile(checkpointer=checkpointer)

print("✅ Multi-agent system created!")
print("Flow: initialize → researcher → [tools] → analyst → writer → END")


In [ ]:
def test_multi_agent_system():
    print("👥 Testing Multi-Agent System")
    print("=" * 50)
    
    config = {"configurable": {"thread_id": "multi_agent_test"}}
    
    # Start the multi-agent workflow
    result = multi_agent_system.invoke(
        {"messages": [HumanMessage(content="Research and analyze the benefits of renewable energy")]},
        config
    )
    
    # Get final state
    final_state = multi_agent_system.get_state(config)
    
    print("📊 Multi-Agent Workflow Complete!")
    print("-" * 40)
    print(f"Task: {final_state.values.get('task', 'Not set')}")
    print(f"Current Agent: {final_state.values.get('current_agent', 'None')}")
    print(f"Task Complete: {final_state.values.get('task_complete', False)}")
    
    print(f"\n🔬 Research Data ({len(final_state.values.get('research_data', []))}):")
    for i, data in enumerate(final_state.values.get('research_data', []), 1):
        print(f"{i}. {data[:100]}...")
    
    print(f"\n📈 Analysis Results ({len(final_state.values.get('analysis_results', []))}):")
    for i, analysis in enumerate(final_state.values.get('analysis_results', []), 1):
        print(f"{i}. {analysis[:100]}...")
    
    print(f"\n📝 Final Report:")
    final_report = final_state.values.get('final_report', 'No report generated')
    print(final_report[:300] + "..." if len(final_report) > 300 else final_report)
    
    print(f"\n💬 Total Messages: {len(final_state.values.get('messages', []))}")
    
    return result

# Test the multi-agent system
multi_agent_result = test_multi_agent_system()
